<a href="https://colab.research.google.com/github/george-kan/fastai/blob/main/Mnist_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#hide
!pip install -Uqq fastbook
import fastbook

     |████████████████████████████████| 727kB 5.7MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 1.2MB 37.8MB/s 
     |████████████████████████████████| 194kB 45.3MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 


In [2]:
#hide
from fastai.vision.all import *
from fastbook import *

In [3]:
path = untar_data(URLs.MNIST)

In [4]:
(path/'training').ls()

(#10) [Path('/root/.fastai/data/mnist_png/training/8'),Path('/root/.fastai/data/mnist_png/training/3'),Path('/root/.fastai/data/mnist_png/training/4'),Path('/root/.fastai/data/mnist_png/training/0'),Path('/root/.fastai/data/mnist_png/training/6'),Path('/root/.fastai/data/mnist_png/training/9'),Path('/root/.fastai/data/mnist_png/training/5'),Path('/root/.fastai/data/mnist_png/training/7'),Path('/root/.fastai/data/mnist_png/training/1'),Path('/root/.fastai/data/mnist_png/training/2')]

In [5]:
folder = 'training/3'

(path/folder).ls()

(#6131) [Path('/root/.fastai/data/mnist_png/training/3/45599.png'),Path('/root/.fastai/data/mnist_png/training/3/46500.png'),Path('/root/.fastai/data/mnist_png/training/3/41404.png'),Path('/root/.fastai/data/mnist_png/training/3/10993.png'),Path('/root/.fastai/data/mnist_png/training/3/983.png'),Path('/root/.fastai/data/mnist_png/training/3/27840.png'),Path('/root/.fastai/data/mnist_png/training/3/17333.png'),Path('/root/.fastai/data/mnist_png/training/3/29097.png'),Path('/root/.fastai/data/mnist_png/training/3/40205.png'),Path('/root/.fastai/data/mnist_png/training/3/39761.png')...]

In [6]:
(path/'testing').ls()

(#10) [Path('/root/.fastai/data/mnist_png/testing/8'),Path('/root/.fastai/data/mnist_png/testing/3'),Path('/root/.fastai/data/mnist_png/testing/4'),Path('/root/.fastai/data/mnist_png/testing/0'),Path('/root/.fastai/data/mnist_png/testing/6'),Path('/root/.fastai/data/mnist_png/testing/9'),Path('/root/.fastai/data/mnist_png/testing/5'),Path('/root/.fastai/data/mnist_png/testing/7'),Path('/root/.fastai/data/mnist_png/testing/1'),Path('/root/.fastai/data/mnist_png/testing/2')]

In [126]:
train_x = []
train_y = []
test_x = []
test_y = []
for digit in range(10):
    folder = 'training/' + str(digit)
    new_inp = [tensor(Image.open(x)) for x in (path/folder).ls()]
    print(digit, len(new_inp))
    train_x += new_inp
    train_y += [digit]*len(new_inp)
    test_folder = 'testing/' + str(digit)
    test_inp = [tensor(Image.open(x)) for x in (path/test_folder).ls()]
    test_x += test_inp
    test_y += [digit]*len(test_inp)
    

0 5923
1 6742
2 5958
3 6131
4 5842
5 5421
6 5918
7 6265
8 5851
9 5949


In [127]:
train_x_stacked = torch.stack(train_x).view(-1, 28*28).float()/255
train_y_stacked = tensor(train_y).view(-1, 1)
train_ds = list(zip(train_x_stacked, train_y_stacked))
train_dl = DataLoader(train_ds, batch_size= 256, shuffle=True)

test_x_stacked = torch.stack(test_x).view(-1, 28*28).float()/255
test_y_stacked = tensor(test_y).view(-1, 1)
test_ds = list(zip(test_x_stacked, test_y_stacked))
test_dl = DataLoader(test_ds, batch_size=256)

dls = DataLoaders(train_dl, test_dl)

train_x_stacked.shape, train_y_stacked.shape, test_x_stacked.shape, test_y_stacked.shape

(torch.Size([60000, 784]),
 torch.Size([60000, 1]),
 torch.Size([10000, 784]),
 torch.Size([10000, 1]))

In [ ]:
train_x[2000]

In [ ]:
df = pd.DataFrame(torch.trunc(tensor(train_x[2000]).float()))
df.style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys')

In [11]:
def batch_accuracy(x, y):
    preds = torch.argmax(x, 1)
    correct = preds == y
    return correct.float().mean()

simple_net = nn.Sequential(
            nn.Linear(28*28, 40),
            nn.ReLU(),
            nn.Linear(40, 40),
            nn.ReLU(),
            nn.Linear(40, 10)
)
loss = CrossEntropyLossFlat()
learn = Learner(dls, simple_net, opt_func = SGD, loss_func = loss, metrics = batch_accuracy)
learn.fit(40, 0.3)


epoch,train_loss,valid_loss,batch_accuracy,time
0,0.342673,0.273259,0.844673,00:01
1,0.211816,0.181899,0.866217,00:01
2,0.167797,0.272501,0.841799,00:01
3,0.144669,0.131778,0.883945,00:01
4,0.122090,0.131966,0.882782,00:01
5,0.103729,0.173585,0.869960,00:01
6,0.090275,0.111314,0.887486,00:01
7,0.082080,0.118197,0.885019,00:01
8,0.081032,0.115382,0.887778,00:01
9,0.074782,0.095189,0.893688,00:01


In [ ]:
x,y = dls.one_batch()
learn.get_preds(dl = [(x,y)])

In [13]:
??Learner

In [160]:
w1 = init_params(28*28, 40)
b1 = init_params(1, 1)
w2 = init_params(40, 10)
#b2 = init_params(1, 1)
lr = 1
params = w1, b1, w2#, b2

In [172]:
def init_params(size1, size2): return torch.randn(size1, size2).requires_grad_()

def my_linear1(x): return x@w1 + b1

def my_linear2(x): return x@w2

def my_relu(x): return torch.maximum(tensor(0), x)

def my_softmax(x): 
    # Normalize the input to avoid numerical problems
    x = x - torch.max(x, dim = 1).values.view(-1, 1)
    return torch.exp(x) / torch.exp(x).sum(dim = 1).unsqueeze(1)

def my_loss(x, y): 
    x = my_softmax(x)
    y = y.view(-1, 1)
    selection = torch.gather(x, 1, y)
    #print(selection)
    return (-torch.log(selection)).mean()

def my_model(x):
    lay1 = my_linear1(x)
    nonlin = my_relu(lay1)
    preds = my_linear2(nonlin)
    return(preds)

def update_grads(preds, y, params):
    loss = my_loss(preds, y)
    print(loss)
    loss.backward()
    for p in params:
        my_grad = p.grad
        p.data -= my_grad*lr
        #print(p.mean())
        #print(my_grad.mean())
        p.grad.zero_()

def eval_batch():
    preds = my_model(test_x_stacked)
    positions = torch.max(preds, dim = 1).indices
    return (positions == test_y_stacked).float().mean()

In [178]:
my_loss(tensor([[1,2,3], [2,3,4]]).float(), tensor([[0],[2]]))

tensor(1.4076)

In [179]:
my_softmax(tensor([[1,2,3], [2,3,4]]).float())

tensor([[0.0900, 0.2447, 0.6652],
        [0.0900, 0.2447, 0.6652]])

In [180]:
-torch.log(tensor([0.0900, 0.6652])).mean()

tensor(1.4078)

In [15]:
inputs, classes = next(iter(train_dl))  

In [ ]:
for i in range(40):
  for x,y in train_dl:
      #print(torch.unique(y, return_counts = True))
      preds = my_model(x)
      update_grads(preds, y, params)
      #print('break')
  #print(eval_batch())